In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
from fastbook import *

Imagenette is a subset of ImageNet consisting of images from 10 categories as opposed to the full 1000 categories of ImageNet. Enables faster iteration, so you can try out more ideas.

In [4]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [5]:
path.ls()

(#2) [Path('/storage/data/imagenette2/val'),Path('/storage/data/imagenette2/train')]

In [7]:
Path.BASE_PATH = path

In [8]:
path.ls()

(#2) [Path('val'),Path('train')]